# Keywords extractor

In [1]:
import pandas as pd

In [2]:
post_df = pd.read_csv("./data/lemmatize_labeled_rusentiment_text_mystem.csv")
post_df.head()

,text,sentiment
0,украина это часть русь искусственно отделять к...,negative
1,говорить относительно небольшой коррупция обра...,negative
2,свой поляк сделать русский понятно литовский п...,neutral
3,фильм зритель любить диковинный соседка пятый ...,neutral
4,государство платить посмотреть денежный поток ...,negative


In [3]:
post_df = post_df.dropna(subset=["text"]).sample(frac=1).reset_index(drop=True)

In [4]:
positive_corpus = post_df[post_df["sentiment"] == "positive"]["text"].tolist()[:10000]
positive_corpus = ". ".join(positive_corpus)

## Rake

In [5]:
from nlp_rake import Rake
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
stops = list(set(stopwords.words("russian")))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dmitry\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
rake = Rake(stopwords=stops, max_words=3)

In [7]:
rake.apply(positive_corpus)[:10]

[('чип тюнер задрачивать', 9.0),
 ('облипливать мочь особенно', 9.0),
 ('жее мооооя любимая', 9.0),
 ('мааааам жи взрослый', 9.0),
 ('кататься жениться кататься', 9.0),
 ('выбирать случаться планировать', 9.0),
 ('одинокий температурить поход', 9.0),
 ('вангуй ловатик зеленый', 9.0),
 ('дугаарыга нь гчих', 9.0),
 ('замечательный профессия предлагать', 9.0)]

## Yake

In [8]:
import yake

In [9]:
extractor = yake.KeywordExtractor(
    lan="ru",
    n=3,
    dedupLim=0.3,
    top=10,
)

In [10]:
extractor.extract_keywords(positive_corpus)

[('размер', 9.760527791958743e-05),
 ('качество хороший', 0.00018670204549712247),
 ('приходить', 0.00021311169844256814),
 ('заказывать', 0.0002463800535873899),
 ('размер подходить качество', 0.00032567728988466545),
 ('доставка', 0.00040676449355744323),
 ('свой', 0.00045572051163252146),
 ('товар', 0.0005696460954690306),
 ('большой', 0.0006096392508756618),
 ('быстро', 0.0006345268009819878)]

## TextRank

In [11]:
from summa import keywords

In [17]:
clean_text_df = pd.read_csv("./data/lemmatize_labeled_rusentiment_text_mystem.csv")

In [18]:
clean_text_df = clean_text_df.dropna(subset=["text"]).sample(frac=1).reset_index(drop=True)

In [19]:
clean_positive_corpus = clean_text_df[clean_text_df["sentiment"] == "positive"]["text"].tolist()[:1000]

In [20]:
clean_positive_corpus = ". ".join(clean_positive_corpus)

In [22]:
keywords.keywords(clean_positive_corpus, language="russian").split("\n")[:10]

['эта',
 'болеть это',
 'свое',
 'приходить',
 'год выходить',
 'день',
 'спасибо',
 'сентябрь годить',
 'хороший',
 'хотеться']